In [49]:
import json
import os
import requests
import fitz  # PyMuPDF for reading PDF files
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from collections import Counter

# Ensure NLTK resources are downloaded
nltk.download('punkt')
nltk.download('stopwords')

# Function to read the JSON dataset
def read_json_dataset(json_path):
    with open(json_path, 'r') as file:
        data = json.load(file)
    return data

# Function to download PDFs
def download_pdf(url, pdf_name):
    print(f"Attempting to download: {url}")  # Print URL before download attempt
    try:
        response = requests.get(url, verify=False)  # Disable SSL verification
        if response.status_code == 200:
            with open(pdf_name, 'wb') as pdf_file:
                pdf_file.write(response.content)
            print(f"Downloaded: {pdf_name}")
            return pdf_name
        else:
            print(f"Failed to download {url} with status code: {response.status_code}")
            return None
    except requests.exceptions.SSLError as ssl_error:
        print(f"SSL Error: {ssl_error}")
        return None
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

# Function to extract text from PDFs
def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:
        text += page.get_text()
    return text

# Function for frequency-based summarization
def frequency_based_summary(text, summary_ratio=0.2):
    stop_words = set(stopwords.words("english"))
    words = word_tokenize(text.lower())
    words = [word for word in words if word.isalnum() and word not in stop_words]

    # Calculate word frequencies
    word_frequencies = Counter(words)

    # Sentence Tokenization
    sentences = sent_tokenize(text)
    sentence_scores = {}

    # Score each sentence based on word frequency
    for sentence in sentences:
        for word in word_tokenize(sentence.lower()):
            if word in word_frequencies:
                if sentence not in sentence_scores:
                    sentence_scores[sentence] = 0
                sentence_scores[sentence] += word_frequencies[word]

    # Sort sentences by score and select top sentences
    num_sentences = int(len(sentences) * summary_ratio)
    summary_sentences = sorted(sentence_scores, key=sentence_scores.get, reverse=True)[:num_sentences]

    # Join selected sentences to form the summary
    return ' '.join(summary_sentences)

# Function for keyword extraction
def keyword_extraction(text, num_keywords=5):
    stop_words = set(stopwords.words("english"))
    words = word_tokenize(text.lower())
    words = [word for word in words if word.isalnum() and word not in stop_words]

    # Calculate word frequencies
    word_frequencies = Counter(words)

    # Extract the most common keywords
    keywords = word_frequencies.most_common(num_keywords)
    return [word for word, _ in keywords]

# Main function to process PDFs from JSON
def main():
    # Load the dataset
    json_path = "C:\\Users\\Hp\\Desktop\\Dataset.json"  # Adjust to your actual path
    data = read_json_dataset(json_path)

    summaries = {}
    all_keywords = {}
    
    # Process each PDF URL
    for key, url in data.items():
        print(f"Processing {key}: {url}")
        # Download the PDF
        pdf_name = f"{key}.pdf"  # Name the downloaded PDF file
        downloaded_pdf = download_pdf(url, pdf_name)

        if downloaded_pdf:
            # Extract text from the downloaded PDF
            text = extract_text_from_pdf(downloaded_pdf)
            if text:
                # Summarize the text
                summary = frequency_based_summary(text)
                summaries[key] = summary
                print(f"Summary for {key}: {summary}")
                
                # Extract keywords
                keywords = keyword_extraction(text)
                all_keywords[key] = keywords
                print(f"Keywords for {key}: {keywords}")

if __name__ == "__main__":
    main()


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Hp\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Hp\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Processing pdf1: https://digiscr.sci.gov.in/pdf_viewer?dir=YWRtaW4vanVkZ2VtZW50X2ZpbGUvanVkZ2VtZW50X3BkZi8xOTUwL3ZvbHVtZSAxL1BhcnQgSS9Db21taXNzaW9uZXIgb2YgSW5jb21lIFRheCwgV2VzdCBCZW5nYWxfQ2FsY3V0dGEgQWdlbmN5IEx0ZC5fMTY5NzYwNjMxMC5wZGY=
Attempting to download: https://digiscr.sci.gov.in/pdf_viewer?dir=YWRtaW4vanVkZ2VtZW50X2ZpbGUvanVkZ2VtZW50X3BkZi8xOTUwL3ZvbHVtZSAxL1BhcnQgSS9Db21taXNzaW9uZXIgb2YgSW5jb21lIFRheCwgV2VzdCBCZW5nYWxfQ2FsY3V0dGEgQWdlbmN5IEx0ZC5fMTY5NzYwNjMxMC5wZGY=


C:\Users\Hp\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'digiscr.sci.gov.in'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


Downloaded: pdf1.pdf
Summary for pdf1: Mitter 
or S.C. Mitter, having been and being still the Manag-
ing Agents of the said Mills have undertaken to 
reimburse the said Mills in respect of the decrees to oe 
made in the said four suits in the manner hereinafter 
appearing NOW THESE PRESENTS WITNESS 
AND IT IS HEREBY AGREED AND DECLARED 
(i) That out of the commission of 3% payable by 
the said Mills to the said Agency under Regulation 131 
of the Articles of Association of the Company, the 
Company shall have paramount lien on and deduct 
and set off a moiety thereof against any payment 
which the said Mills may make in respect of the decrees 
or any of them and/or costs of the said suits. Therefore this case is covered 
by the judgment of the Court of Appeal in Mitchell's 
case(') ...... " Applying this line of reasoning the High 
Court differed from the conclusion of the Tribunal and 
allowed the deduction to the respondent company under 
section 10(2) (xv) of the Income-tax Act, as

C:\Users\Hp\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'digiscr.sci.gov.in'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


...

Downloaded: pdf16.pdf
Summary for pdf16: The order log and trade 
log revealed that there was synchronization of buying and selling orders 
by these brokers as under: 
 
Date 
Trade 
Time 
Tr 
No 
Order NO 
Order time 
Diff  
(sec)  B/S 
Mem Client Code 
Qty  
Price 
27-02-01 12:16:13 
1 
61175322 
12:16:13 
**** buy 
35 rathyatrasa 
10000 
335.00 
27-02-01 12:16:13 
1 
31317843 
12:15:37 
0:00:36 sell 
545 b110 
10000 
335.00 
27-02-01 12:16:56 
2 
61175333 
12:16:56 
**** buy 
35 rathyatrasa 
10000 
335.00 
27-02-01 12:16:56 
2 
31317859 
12:16:32 
0:00:24 sell 
545 b110 
10000 
335.00 
27-02-01 12:17:18 
3 
61175338 
12:17:18 
**** buy 
35 rathyatrasa 
10000 
335.00 
27-02-01 12:17:18 
3 
31317864 
12:17:07 
0:00:11 sell 
545 b110 
10000 
335.00 
27-02-01 12:17:27 
4 
61175342 
12:17:27 
**** buy 
35 rathyatrasa 
10000 
335.00 
27-02-01 12:17:27 
4 
31317867 
12:17:17 
0:00:10 sell 
545 b110 
10000 
335.00 
27-02-01 12:17:33 
5 
61175347 
12:17:33 
**** buy 
35 rathyatrasa 
10000 
3

C:\Users\Hp\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ijtr.nic.in'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


Downloaded: pdf17.pdf
Summary for pdf17: …………  
Signature 
 
Appendix – 5 
izk:i la[;k 45 
 
Fkkus esa U;k;ky; ds Hkkjlk/kd vf/kdkjh ds le{k 
gkftj gksus ds fy, ca/ki= vkSj tekuri= ¼/kkjk 
436] 436d] 437] 437d] 438¼3½] vkSj 441 nsf[k,½ 
 
izsf"krh ------------------ 
 
 
eSa ¼uke½ ¼LFkku½ dk fuolh gWw -------------
--------- Fkkus ds Hkkjlk/kd vf/kdkjh }kjk] fcuk 
okj.V  fxjQrkj ;k fu:) dj fy, tkus ij ¼;k ------
------- U;k;ky; ds le{k yk, tkus ij½ vijk/k ls 
vkjksfir fd;k x;k gWw rFkk eq>ls ,sls vf/kdkjh 
;k U;k;ky; ds le{k ,sls izR;sd fnu] gkftj 
52 
 
gksÅaxk] ftlesa ,sls vkjksi ds ckcr dksbZ 
vUos"k.k 
;k 
fopkj.k 
fd;k 
tk,] 
rFkk 
ijh{k.kksijkUr 
,oa 
fu.kZ;ksijkUr 
,sls 
mPprj 
U;k;ky; ds le{k] ftlds }kjk fu.kZ; dh frfFk ls 6 
ekg dh vof/k ds Hkhrj ;fn fu.kZ; ds fo:) nkf[ky 
fdlh vihy ;k fiVh’ku esa uksfVl tkjh fd;k tkrk 
gS] mifLFkr gksÅaxk rFkk eSa vius dks vkc) djrk 
gWw fd ;fn blls pwd d:W rks esjh ---------------
-- jkf’k ljdkj dks le"kg`r gks tk;sxhA 
rk0------------------

C:\Users\Hp\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'enforcementdirectorate.gov.in'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


SSL Error: [SSL: DECRYPTION_FAILED_OR_BAD_RECORD_MAC] decryption failed or bad record mac (_ssl.c:2580)


In [58]:
import json
import os
import requests
import fitz  # PyMuPDF for reading PDF files
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from collections import Counter
from pymongo import MongoClient
from concurrent.futures import ThreadPoolExecutor
import time

# Ensure NLTK resources are downloaded
nltk.download('punkt')
nltk.download('stopwords')

# MongoDB setup
client = MongoClient('mongodb://localhost:27017/')  # Adjust your MongoDB URI as needed
db = client['pdf_processing']  # Replace with your database name
collection = db['documents']  # Replace with your collection name

# Function to read the JSON dataset
def read_json_dataset(json_path):
    with open(json_path, 'r') as file:
        data = json.load(file)
    return data

# Function to download PDFs
def download_pdf(url, pdf_name):
    print(f"Attempting to download: {url}")  # Print URL before download attempt
    try:
        response = requests.get(url, verify=False)  # Disable SSL verification
        if response.status_code == 200:
            with open(pdf_name, 'wb') as pdf_file:
                pdf_file.write(response.content)
            print(f"Downloaded: {pdf_name}")
            return pdf_name
        else:
            print(f"Failed to download {url} with status code: {response.status_code}")
            return None
    except requests.exceptions.SSLError as ssl_error:
        print(f"SSL Error: {ssl_error}")
        return None
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

# Function to extract text from PDFs
def extract_text_from_pdf(pdf_path):
    try:
        doc = fitz.open(pdf_path)
        text = ""
        for page in doc:
            text += page.get_text()
        return text
    except Exception as e:
        print(f"Error extracting text from {pdf_path}: {e}")
        return None

# Function for frequency-based summarization
def frequency_based_summary(text, summary_ratio=0.2):
    stop_words = set(stopwords.words("english"))
    words = word_tokenize(text.lower())
    words = [word for word in words if word.isalnum() and word not in stop_words]

    # Calculate word frequencies
    word_frequencies = Counter(words)

    # Sentence Tokenization
    sentences = sent_tokenize(text)
    sentence_scores = {}

    # Score each sentence based on word frequency
    for sentence in sentences:
        for word in word_tokenize(sentence.lower()):
            if word in word_frequencies:
                if sentence not in sentence_scores:
                    sentence_scores[sentence] = 0
                sentence_scores[sentence] += word_frequencies[word]

    # Sort sentences by score and select top sentences
    num_sentences = int(len(sentences) * summary_ratio)
    summary_sentences = sorted(sentence_scores, key=sentence_scores.get, reverse=True)[:num_sentences]

    # Join selected sentences to form the summary
    return ' '.join(summary_sentences)

# Function for keyword extraction
def keyword_extraction(text, num_keywords=5):
    stop_words = set(stopwords.words("english"))
    words = word_tokenize(text.lower())
    words = [word for word in words if word.isalnum() and word not in stop_words]

    # Calculate word frequencies
    word_frequencies = Counter(words)

    # Extract the most common keywords
    keywords = word_frequencies.most_common(num_keywords)
    return [word for word, _ in keywords]

# Function to store document metadata in MongoDB
def store_metadata(pdf_name, pdf_path):
    metadata = {
        'name': pdf_name,
        'path': pdf_path,
        'size': os.path.getsize(pdf_path)  # Get the file size
    }
    collection.insert_one(metadata)
    print(f"Stored metadata for {pdf_name}")

# Function to update MongoDB with summary and keywords
def update_document(key, summary, keywords):
    update_query = {'name': key}
    new_values = {"$set": {"summary": summary, "keywords": keywords}}
    collection.update_one(update_query, new_values)
    print(f"Updated {key} with summary and keywords.")

# Function to process a single PDF
def process_pdf(pdf_path):
    pdf_name = os.path.basename(pdf_path)  # Get the file name
    print(f"Processing {pdf_name}...")  # Log when processing starts
    store_metadata(pdf_name, pdf_path)  # Store metadata in MongoDB

    start_time = time.time()  # Start timer

    # Extract text from the downloaded PDF
    text = extract_text_from_pdf(pdf_path)
    
    if text:
        print(f"Extracted text from {pdf_name}")  # Log when text extraction is successful

        # Summarize the text
        summary = frequency_based_summary(text)
        print(f"Summary for {pdf_name}: {summary}")
        
        # Extract keywords
        keywords = keyword_extraction(text)
        print(f"Keywords for {pdf_name}: {keywords}")

        # Update MongoDB with summary and keywords
        update_document(pdf_name, summary, keywords)
    else:
        print(f"No text extracted from {pdf_name}.")  # Log if no text was extracted

    end_time = time.time()  # End timer
    print(f"Processed {pdf_name} in {end_time - start_time:.2f} seconds.")

# Function to process all PDFs in a specified folder
def process_pdfs_in_folder(folder_path):
    # List all PDF files in the specified folder
    pdf_files = [f for f in os.listdir(folder_path) if f.endswith('.pdf')]
    
    print(f"Found PDF files: {pdf_files}")  # Print the list of found PDF files
    
    with ThreadPoolExecutor() as executor:
        for pdf_file in pdf_files:
            pdf_path = os.path.join(folder_path, pdf_file)
            print(f"Submitting {pdf_path} for processing...")  # Log which PDF is being processed
            executor.submit(process_pdf, pdf_path)  # Submit each PDF for processing

# Main function to execute the PDF processing pipeline
def main():
    folder_path = r"C:\Users\Hp\Desktop\DownloadedPDFs"  # Adjust to your actual folder path
    process_pdfs_in_folder(folder_path)

if __name__ == "__main__":
    main()


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Hp\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Hp\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Found PDF files: ['1292585113260.pdf', '20240716890312078.pdf', '250883_english_01042024.pdf']
Submitting C:\Users\Hp\Desktop\DownloadedPDFs\1292585113260.pdf for processing...
Processing 1292585113260.pdf...
Submitting C:\Users\Hp\Desktop\DownloadedPDFs\20240716890312078.pdf for processing...
Processing 20240716890312078.pdf...
Submitting C:\Users\Hp\Desktop\DownloadedPDFs\250883_english_01042024.pdf for processing...
Processing 250883_english_01042024.pdf...
